In [ ]:
# Run this cell to install all necessary libraries
!pip install "ibm-watsonx-ai>=0.2.0" pypdf2 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 2.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 10.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.1 MB/s eta 0:00:00


In [ ]:
import PyPDF2
import io
from google.colab import files
from ibm_watsonx_ai.foundation_models import ModelInference
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams
from google.colab import userdata

In [ ]:

try:
    IBM_CLOUD_API_KEY = userdata.get('IBM_CLOUD_API_KEY')
    WATSONX_PROJECT_ID = userdata.get('WATSONX_PROJECT_ID')
except Exception as e:
    print("Error: Could not find Secrets. Did you add 'IBM_CLOUD_API_KEY' and 'WATSONX_PROJECT_ID' to the Colab Secrets (🔑) panel?")

model_id = "mistralai/mistral-small-3-1-24b-instruct-2503"
decoding_method = "greedy"
max_new_tokens = 15
min_new_tokens = 1

JOB_CATEGORIES = [
    "Data Science",
    "Software Development",
    "HR (Human Resources)",
    "Project Management",
    "Cybersecurity"
]

def initialize_watsonx_model(api_key, project_id):
    """Initializes the Watsonx.ai model client."""
    credentials = {
        "url": "https://us-south.ml.cloud.ibm.com",
        "apikey": api_key
    }

    try:
        model = ModelInference(
            model_id=model_id,
            params=None, # We'll set params in the generate call
            credentials=credentials,
            project_id=project_id
        )
        print(f"✅ IBM Watsonx.ai model initialized successfully ({model_id}).")
        return model
    except Exception as e:
        print(f"❌ Error initializing Watsonx model: {e}")
        return None

In [ ]:
def extract_text_from_pdf(pdf_bytes):
    try:
        pdf_file = io.BytesIO(pdf_bytes)
        pdf_reader = PyPDF2.PdfReader(pdf_file)
        text = ""
        for page in pdf_reader.pages:
            text += page.extract_text()
        print("✅ PDF text extracted successfully.")
        return text
    except Exception as e:
        print(f"❌ Error reading PDF: {e}")
        return None

def classify_resume_with_watsonx(model, resume_text, categories):
    category_string = ", ".join(categories)

    prompt = f"""
    [INST] You are an expert HR recruiter. Read the following resume text and classify it into exactly one
    of the following job categories: {category_string}.
    Return ONLY the name of the most relevant category and nothing else.

    Resume Text:
    ---
    {resume_text}
    ---
    Category: [/INST]
    """

    params = {
        GenParams.DECODING_METHOD: decoding_method,
        GenParams.MAX_NEW_TOKENS: max_new_tokens,
        GenParams.MIN_NEW_TOKENS: min_new_tokens,
    }

    print("⏳ Sending resume to IBM Watsonx.ai for analysis...")

    try:
        classification = model.generate_text(prompt=prompt, params=params)
        classification = classification.strip()

        if classification:
            for cat in categories:
                if cat.lower() in classification.lower():
                    print("✅ Classification received.")
                    return cat

            print(f"⚠️ Model returned an unexpected value: {classification}")
            return classification
        else:
            return "Error: No response from model."

    except Exception as e:
        return f"Error during API call: {e}"

In [ ]:
model = initialize_watsonx_model(IBM_CLOUD_API_KEY, WATSONX_PROJECT_ID)

if model:
    print("\nPlease upload your resume PDF...")
    uploaded = files.upload()

    if not uploaded:
        print("No file uploaded. Please run the cell again.")
    else:
        file_name = list(uploaded.keys())[0]
        file_bytes = uploaded[file_name]

        print(f"\nProcessing file: {file_name}...")
        resume_text = extract_text_from_pdf(file_bytes)

        if resume_text:
            classification = classify_resume_with_watsonx(model, resume_text, JOB_CATEGORIES)

            print("\n" + "="*30)
            print("  CLASSIFICATION RESULT")
            print("="*30)
            print(f"📄 Resume: {file_name}")
            print(f"🤖 AI Classification: {classification}")
            print("="*30)

✅ IBM Watsonx.ai model initialized successfully (mistralai/mistral-small-3-1-24b-instruct-2503).

Please upload your resume PDF...


Saving DS Resume.pdf to DS Resume.pdf

Processing file: DS Resume.pdf...
✅ PDF text extracted successfully.
⏳ Sending resume to IBM Watsonx.ai for analysis...
✅ Classification received.

  CLASSIFICATION RESULT
📄 Resume: DS Resume.pdf
🤖 AI Classification: Data Science
